In [1]:
import pandas as pd

In [2]:
arquivo = "senadores_full"
#deputados
arquivo_dep = "Ano-2017.csv"

In [3]:
arq_sena = pd.read_csv(arquivo, delimiter=";", encoding='UTF-8',header=0,index_col = 0)

arq_depu = pd.read_csv(arquivo_dep, delimiter=";", encoding='UTF-8',header=0)

In [4]:
#Drop nos deputados
arq_depu.drop(['idecadastro','nuCarteiraParlamentar','nuLegislatura','codLegislatura','numSubCota','numEspecificacaoSubCota','txtDescricaoEspecificacao','txtNumero','indTipoDocumento','vlrDocumento','vlrGlosa','numParcela','txtPassageiro','txtTrecho','numLote','numRessarcimento','vlrRestituicao','nuDeputadoId','ideDocumento'],axis=1, inplace=True)

#Renomeando colunas
arq_depu.rename(columns={'txNomeParlamentar':'NOME',
                         'sgUF':'ESTADO',
                         'sgPartido':'PARTIDO',
                         'txtDescricao':'TIPO_DESPESA',
                         'txtFornecedor':'FORNECEDOR',
                         'txtCNPJCPF':'CNPJ_CPF',
                         'datEmissao':'DATA',
                         'vlrLiquido':'VALOR',
                         'numMes':'MES',
                         'numAno':'ANO'}, inplace=True)

#TRATANDO COLUNAS

#trocando virgula por ponto
arq_depu['VALOR']=arq_depu['VALOR'].str.replace(',','.')

#tranformando em float
arq_depu['VALOR']=arq_depu['VALOR'].astype(float)

#valor absoluto para a coluna valor
arq_depu['VALOR']=arq_depu['VALOR'].abs()

#mudar formato da data e trunca os segundos
#arq_depu['DATA']=pd.to_datetime(arq_depu['DATA'], errors ='coerce').dt.date

#tranformando pra n64
arq_depu['DATA']=pd.to_datetime(arq_depu['DATA'],errors ='coerce')

#adicionando a coluna CARGO pra marcar que ele é deputado
arq_depu.insert(loc=0, column='CARGO', value='D')

#Inserindo coluna CAT_DESPESA
arq_depu.insert(loc=0, column='CAT_DESPESA', value='0')

#reordenando
arq_depu = arq_depu[['ESTADO','PARTIDO','CARGO','NOME','VALOR','TIPO_DESPESA','CNPJ_CPF','FORNECEDOR','CAT_DESPESA','MES','ANO','DATA']]


In [5]:
arq_depu.dtypes

ESTADO                  object
PARTIDO                 object
CARGO                   object
NOME                    object
VALOR                  float64
TIPO_DESPESA            object
CNPJ_CPF               float64
FORNECEDOR              object
CAT_DESPESA             object
MES                      int64
ANO                      int64
DATA            datetime64[ns]
dtype: object

In [6]:
arq_depu.head()

,ESTADO,PARTIDO,CARGO,NOME,VALOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,CAT_DESPESA,MES,ANO,DATA
0,RR,DEM,D,ABEL MESQUITA JR.,296.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,1.213285e+13,WM PAPELARIA E ESCRITÓRIO,0,4,2017,2017-04-26
1,RR,DEM,D,ABEL MESQUITA JR.,175.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,1.213285e+13,WMS COMERCIO DE ARTIGOS DE PAPELARIA LTDA - ME,0,10,2017,2017-10-05
2,RR,DEM,D,ABEL MESQUITA JR.,100.0,COMBUSTÍVEIS E LUBRIFICANTES.,6.924180e+11,AUTO POSTO CINCO ESTRELAS LTDA,0,10,2017,2017-10-23
3,RR,DEM,D,ABEL MESQUITA JR.,100.0,COMBUSTÍVEIS E LUBRIFICANTES.,3.482208e+12,Auto Posto JK Ltda.,0,5,2017,2017-05-03
4,RR,DEM,D,ABEL MESQUITA JR.,150.0,COMBUSTÍVEIS E LUBRIFICANTES.,3.482208e+12,Auto Posto JK Ltda.,0,5,2017,2017-05-08


In [7]:
arq_depu.pivot_table(index='CAT_DESPESA')

,ANO,CNPJ_CPF,MES,VALOR
CAT_DESPESA,,,,
0,2017.0,1.334529e+13,6.531557,727.542415


In [8]:
# Função para classificar as depesas dos deputados

def classifica_despesa_dep (row):
    if row['TIPO_DESPESA'] == 'ASSINATURA DE PUBLICAÇÕES' :
        return '2'
    if row['TIPO_DESPESA'] == 'COMBUSTÍVEIS E LUBRIFICANTES.' :
        return '5'
    if row['TIPO_DESPESA'] == 'CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.' :
        return '3'
    if row['TIPO_DESPESA'] == 'DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.' :
        return '4'
    if row['TIPO_DESPESA'] == 'Emissão Bilhete Aéreo' :
        return '6'
    if row['TIPO_DESPESA'] == 'FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR' :
        return '5'
    if row['TIPO_DESPESA'] == 'HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.' :
        return '5'
    if row['TIPO_DESPESA'] == 'LOCAÇÃO OU FRETAMENTO DE AERONAVES' :
        return '5'
    if row['TIPO_DESPESA'] == 'LOCAÇÃO OU FRETAMENTO DE EMBARCAÇÕES' :
        return '5'
    if row['TIPO_DESPESA'] == 'LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES' :
        return '5'
    if row['TIPO_DESPESA'] == 'MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR' :
        return '1'
    if row['TIPO_DESPESA'] == 'PARTICIPAÇÃO EM CURSO, PALESTRA OU EVENTO SIMILAR' :
        return '5'
    if row['TIPO_DESPESA'] == 'PASSAGENS AÉREAS' :
        return '6'
    if row['TIPO_DESPESA'] == 'PASSAGENS TERRESTRES, MARÍTIMAS OU FLUVIAIS' :
        return '6'
    if row['TIPO_DESPESA'] == 'SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.' :
        return '7'
    if row['TIPO_DESPESA'] == 'SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO' :
        return '5'
    if row['TIPO_DESPESA'] == 'SERVIÇOS POSTAIS' :
        return '2'
    if row['TIPO_DESPESA'] == 'TELEFONIA' :
        return '1'
    return '0'

#aplicar a função em todas as linhas
arq_depu['CAT_DESPESA'] = arq_depu.apply (lambda row: classifica_despesa_dep(row),axis=1)

In [9]:
#pular
arq_depu.pivot_table(index='NOME',aggfunc=sum)

,ANO,CNPJ_CPF,MES,VALOR
NOME,,,,
ABEL MESQUITA JR.,883446,2.896754e+15,2706,624646.00
ADAIL CARNEIRO,687797,4.761549e+15,2209,517959.49
ADALBERTO CAVALCANTI,831004,3.540093e+15,2525,514862.54
ADELMO CARNEIRO LEÃO,2073476,1.010580e+16,6861,437764.48
ADELSON BARRETO,1058925,2.460207e+15,3521,455995.23
ADEMIR CAMILO,1530903,9.764938e+15,4619,446492.20
ADILTON SACHETTI,756375,3.346876e+15,2242,245893.43
ADÉRMIS MARINI,1157758,8.021506e+15,1882,175274.60
AELTON FREITAS,1028670,4.987389e+15,2880,438134.79


In [10]:
#unindo as planilhas
frames =[arq_sena, arq_depu]

result_final = pd.concat(frames)
#: frames = [df1, df2, df3]
#In [5]: result = pd.concat(frames)

In [11]:
result_final.head()

,ESTADO,PARTIDO,CARGO,NOME,VALOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,CAT_DESPESA,MES,ANO,DATA
0,RO,PDT,S,ACIR GURGACZ,97.00,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ELETROBRÁS DISTRIBUIÇÃO RONDÔNIA,1,1,2017,2017-01-18
1,RO,PDT,S,ACIR GURGACZ,6000.00,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,1,1,2017,2017-01-17
2,RO,PDT,S,ACIR GURGACZ,418.04,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,1,1,2017,2017-01-18
3,RO,PDT,S,ACIR GURGACZ,1958.95,"Passagens aéreas, aquáticas e terrestres nacio...",13.419.034/0001-67,e-Destinos.com.br,6,1,2017,2017-01-23
4,RO,PDT,S,ACIR GURGACZ,1178.67,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,6,1,2017,2017-01-24


In [12]:
print('result_final dataset possui %s linhas e %s colunas' % (result_final.shape[0],result_final.shape[1]))
print('arq_sena possui %s linhas e %s colunas' % (arq_sena.shape[0],arq_sena.shape[1]))
print('arq_depu possui %s linhas e %s colunas' % (arq_depu.shape[0],arq_depu.shape[1]))

result_final dataset possui 368382 linhas e 12 colunas
arq_sena possui 26957 linhas e 12 colunas
arq_depu possui 341425 linhas e 12 colunas


In [13]:
#organizando por ordem alfabetica
result_final = result_final.sort_values('NOME')

In [14]:
result_final.head()

,ESTADO,PARTIDO,CARGO,NOME,VALOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,CAT_DESPESA,MES,ANO,DATA
0,RR,DEM,D,ABEL MESQUITA JR.,296.00,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,1.21329e+13,WM PAPELARIA E ESCRITÓRIO,1,4,2017,2017-04-26 00:00:00
299,RR,DEM,D,ABEL MESQUITA JR.,691.93,Emissão Bilhete Aéreo,7.57565e+12,Cia Aérea - GOL,6,9,2017,2017-09-28 00:00:00
298,RR,DEM,D,ABEL MESQUITA JR.,674.80,Emissão Bilhete Aéreo,7.57565e+12,Cia Aérea - GOL,6,3,2017,2017-03-14 00:00:00
297,RR,DEM,D,ABEL MESQUITA JR.,674.80,Emissão Bilhete Aéreo,7.57565e+12,Cia Aérea - GOL,6,2,2017,2017-02-20 00:00:00
296,RR,DEM,D,ABEL MESQUITA JR.,508.93,Emissão Bilhete Aéreo,7.57565e+12,Cia Aérea - GOL,6,11,2017,2017-11-24 00:00:00


In [15]:
#salvar arquivo
result_final.to_csv('legislativo', sep=';', encoding='utf-8')